In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json 
import time

In [9]:
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By

In [161]:
s = Service('C://Users//hantt3//Downloads//chromedriver_win32')
driver = webdriver.Chrome(service=s)

**Handle pagination using xpath**

In [162]:
def getCoinInfo(link):
    driver.get(link)
    count = 1
    rank, name, price, market_cap, volumn, circulating_supply = [], [], [], [], [],[]
    while True:
        try:
            print("Crawl Page " + str(count))
            screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
            i = 1

            while True:
                # scroll one screen height each time
                driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
                i += 1
                time.sleep(5)
                # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
                scroll_height = driver.execute_script("return document.body.scrollHeight;")  
                # Break the loop when the height we need to scroll to is larger than the total scroll height
                if (screen_height) * i > scroll_height:
                    break 
            
            rank1 =  driver.find_elements(By.XPATH, '//table[@class="sc-beb003d5-3 ieTeVa cmc-table  "]/tbody/tr/td[2]')
            rank =  [s.text for s in rank1] + rank
            name1 =  driver.find_elements(By.XPATH, '//table[@class="sc-beb003d5-3 ieTeVa cmc-table  "]/tbody/tr/td[3]')
            name =  [s.text for s in name1] + name
            price1 =  driver.find_elements(By.XPATH, '//table[@class="sc-beb003d5-3 ieTeVa cmc-table  "]/tbody/tr/td[4]')
            price = [s.text for s in price1] + price
            market_cap1 = driver.find_elements(By.XPATH, '//table[@class="sc-beb003d5-3 ieTeVa cmc-table  "]/tbody/tr/td[8]')
            market_cap =  [s.text for s in market_cap1] + market_cap
            volumn1 =  driver.find_elements(By.XPATH, '//table[@class="sc-beb003d5-3 ieTeVa cmc-table  "]/tbody/tr/td[9]')
            volumn =  [s.text for s in volumn1] + volumn
            circulating_supply1 =  driver.find_elements(By.XPATH, '//table[@class="sc-beb003d5-3 ieTeVa cmc-table  "]/tbody/tr/td[10]')
            circulating_supply =  [s.text for s in circulating_supply1] + circulating_supply
    
            next_pagination_cmt = driver.find_element("xpath", "/html/body/div[1]/div/div[1]/div[2]/div/div[1]/div[6]/div[1]/div/ul/li[10]/a")
            next_pagination_cmt.click()
            print("Clicked on button next page!")
            time.sleep(5)
            count += 1
            #scrape only 2 page
            if count == 3:
                break
        except ElementNotInteractableException:
            print("Element Not Interactable Exception!")
            break
    print(len(rank))
    coinmarketcap1 = pd.DataFrame(list(zip(rank, name, price, market_cap, volumn, circulating_supply)),
                        columns=['rank', 'name', 'price', 'market_cap', 'volumn (24h)', 'circulating_supply']) 
    return coinmarketcap1

In [163]:
url = 'https://coinmarketcap.com/'
df = getCoinInfo(url)
driver.close()

Crawl Page 1
Clicked on button next page!
Crawl Page 2
Clicked on button next page!
200


In [164]:
df.shape

(200, 6)

In [165]:
df.head()

,rank,name,price,market_cap,volumn (24h),circulating_supply
0,101,Nexo\nNEXO,$0.6625,"$371,038,730","$3,672,399\n5,539,306 NEXO","560,000,011 NEXO"
1,102,Oasis Network\nROSE,$0.06167,"$354,403,099","$39,448,613\n639,830,432 ROSE","5,725,718,533 ROSE"
2,103,Ethereum Name Service\nENS,$12.00,"$340,417,782","$36,260,187\n3,020,878 ENS","28,337,389 ENS"
3,104,Mask Network\nMASK,$4.54,"$336,683,254","$100,485,135\n22,142,229 MASK","74,162,500 MASK"
4,105,NEM\nXEM,$0.0373,"$335,597,571","$12,054,369\n323,184,366 XEM","8,999,999,999 XEM"


In [172]:
df['rank']= df['rank'].astype(int)

In [173]:
df = df.sort_values(by='rank', ascending = True)

In [175]:
df.set_index('rank', inplace = True)

In [176]:
df.head()

,name,price,market_cap,volumn (24h),circulating_supply
rank,,,,,
1,Bitcoin\nBTC,"$27,264.27","$527,851,684,194","$18,283,337,877\n670,700 BTC","19,353,137 BTC"
2,Ethereum\nETH,"$1,852.80","$223,159,002,459","$10,427,046,453\n5,627,476 ETH","120,414,184 ETH"
3,Tether\nUSDT,$1.00,"$81,443,835,274","$29,854,228,898\n29,840,627,074 USDT","81,421,868,893 USDT"
4,BNB\nBNB,$323.42,"$50,438,075,187","$833,761,378\n2,577,950 BNB","155,864,509 BNB"
5,USD Coin\nUSDC,$1.00,"$30,786,259,560","$5,068,852,303\n5,067,624,031 USDC","30,779,107,463 USDC"


**Handle page pagination using brute force method.**

In [147]:
page = 1
rank, name, price, market_cap, volumn, circulating_supply = [], [], [], [], [], []
while(page):
    print("Crawl Page " + str(page))
    url = 'https://coinmarketcap.com/?page={}'.format(page)
    driver.get(url)
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(5)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break 
            
    rank1 =  driver.find_elements(By.XPATH, '//table[@class="sc-beb003d5-3 ieTeVa cmc-table  "]/tbody/tr/td[2]')
    rank =  [s.text for s in rank1] + rank
    name1 =  driver.find_elements(By.XPATH, '//table[@class="sc-beb003d5-3 ieTeVa cmc-table  "]/tbody/tr/td[3]')
    name =  [s.text for s in name1] + name
    price1 =  driver.find_elements(By.XPATH, '//table[@class="sc-beb003d5-3 ieTeVa cmc-table  "]/tbody/tr/td[4]')
    price = [s.text for s in price1] + price
    market_cap1 = driver.find_elements(By.XPATH, '//table[@class="sc-beb003d5-3 ieTeVa cmc-table  "]/tbody/tr/td[8]')
    market_cap =  [s.text for s in market_cap1] + market_cap 
    volumn1 =  driver.find_elements(By.XPATH, '//table[@class="sc-beb003d5-3 ieTeVa cmc-table  "]/tbody/tr/td[9]')
    volumn =  [s.text for s in volumn1] + volumn
    circulating_supply1 =  driver.find_elements(By.XPATH, '//table[@class="sc-beb003d5-3 ieTeVa cmc-table  "]/tbody/tr/td[10]')
    circulating_supply =  [s.text for s in circulating_supply1] + circulating_supply
    page += 1
    #scrape only 2 page
    if page == 3:
        break
coinmarketcap = pd.DataFrame(list(zip(rank, name, price, market_cap, volumn, circulating_supply)),
                        columns=['rank', 'name', 'price', 'market_cap', 'volumn (24h)', 'circulating_supply']) 

    
    
    

Crawl Page 1
Crawl Page 2


In [178]:
coinmarketcap['rank']= coinmarketcap['rank'].astype(int)

In [179]:
coinmarketcap = coinmarketcap.sort_values(by='rank', ascending = True)

In [180]:
coinmarketcap.set_index('rank', inplace = True)
coinmarketcap.head()

,name,price,market_cap,volumn (24h),circulating_supply
rank,,,,,
1,Bitcoin\nBTC,"$27,240.88","$527,262,018,086","$18,292,335,492\n671,136 BTC","19,353,137 BTC"
2,Ethereum\nETH,"$1,851.52","$222,966,693,444","$10,449,060,510\n5,641,282 ETH","120,414,184 ETH"
3,Tether\nUSDT,$1.00,"$81,472,729,418","$29,945,477,047\n29,936,950,199 USDT","81,421,868,893 USDT"
4,BNB\nBNB,$323.29,"$50,384,724,518","$839,671,715\n2,597,290 BNB","155,864,509 BNB"
5,USD Coin\nUSDC,$1.00,"$30,793,221,621","$5,088,021,347\n5,087,776,163 USDC","30,779,107,463 USDC"
